Best Response Sequences in the Iterated Prisoner's Dilemma
------------

This Chapter explores best response strategies, to a collection of opponents, in the form of sequences. The APL project is used to simulate matches between these opponents and the "sequence" players. Morespecifically, the class `Cycler` is used to simulate the sequences.

In [ ]:
import axelrod as axl

**Example of $S = \{D, D, D, C, C, C, D, D, C, C\}$ against Cooperator**

In [ ]:
players = [axl.Cycler('DDDCCCDDCC'), axl.Cooperator()]

In [ ]:
match = axl.Match(players, turns=10)
match.play()

In [ ]:
match.final_score_per_turn()

**Example of$S = \{D, D, D, C, C, C, D, D, C, C\}$ against Tit For Tat**

In [ ]:
players = [axl.Cycler('DDDCCCDDCC'), axl.TitForTat()]

In [ ]:
match = axl.Match(players, turns=10)
match.play()

In [ ]:
match.final_score_per_turn()

**Example against Random**

In [ ]:
players = [axl.Random(), axl.Cycler('DDDCCCDDCC')]

In [ ]:
axl.seed(1)
match = axl.Match(players, turns=10)
actions = match.play()

In [ ]:
actions

In [ ]:
match.final_score_per_turn()

In [ ]:
axl.seed(2)
match = axl.Match(players, turns=10)
actions = match.play()

In [ ]:
match.final_score_per_turn()

This work uses random seeds to capture different behaviours of stochastic strategies. Random seeding also makes it possible to reproduce the behaviour of stochastic strategies.

In [ ]:
players = [axl.Cycler('DDDCCCDDCC'), axl.Random()]
for seed in range(5):
    axl.seed(seed)
    match = axl.Match(players, turns=10)
    actions = match.play()
    print(actions, match.final_score_per_turn())
    print("================================================================================")


`sequence_sensei` is a package which has been developed to perform a genetic algorithm on sequences.

In [ ]:
import sequence_sensei as ss
import numpy as np

Generating the starting population is done using the following code:

In [ ]:
initial_population = ss.get_initial_population(half_size_of_population=5, sequence_length=8)

In [ ]:
np.matrix(initial_population)

In [ ]:
np.matrix([[axl.Action(gene) for gene in member] for member in initial_population])

`sequence_sensei` includes functions for the crossover and mutation properties of a genetic algorithm.

In [ ]:
import random
import sequence_sensei as ss

In [ ]:
turns = 10

cooperator = [1 for _ in range(turns)]
alternator = [i % 2 for i in range(turns)]

In [ ]:
random.seed(1)
new_member = ss.crossover(cooperator, alternator)

In [ ]:
new_member

In [ ]:
random.seed(1)
[ss.mutation(gene, mutation_probability=0.05) for gene in new_member] 